In [29]:
import tensorflow as tf
from tensorflow.keras import layers
import os, shutil, zipfile

In [30]:
ZIP_FILE = "brain_mri.zip"
if not os.path.exists(ZIP_FILE):
    raise FileNotFoundError("Put 'brain_mri.zip' next to this notebook!")

print("Unzipping...")
with zipfile.ZipFile(ZIP_FILE, 'r') as z:
    z.extractall("temp_data")
print("Unzipped to 'temp_data/'")

Unzipping...
Unzipped to 'temp_data/'


In [31]:

BINARY_DIR = "brain_binary"
os.makedirs(f"{BINARY_DIR}/no", exist_ok=True)
os.makedirs(f"{BINARY_DIR}/yes", exist_ok=True)

def copy_images(src, dst):
    if os.path.exists(src):
        for img in os.listdir(src):
            if img.lower().endswith(('.png', '.jpg', '.jpeg')):
                shutil.copy(f"{src}/{img}", f"{dst}/{img}")

# ---- no tumor → no/ ----
for split in ["Training", "Testing"]:
    copy_images(f"temp_data/{split}/notumor", f"{BINARY_DIR}/no")

# ---- tumors → yes/ ----
tumor_types = ["glioma", "meningioma", "pituitary"]
for split in ["Training", "Testing"]:
    for t in tumor_types:
        copy_images(f"temp_data/{split}/{t}", f"{BINARY_DIR}/yes")

# ---- RESULT ----
print(f"no/  → {len(os.listdir(f'{BINARY_DIR}/no'))} images")
print(f"yes/ → {len(os.listdir(f'{BINARY_DIR}/yes'))} images")

no/  → 2000 images
yes/ → 5023 images


In [32]:
class TransformerBlock(layers.Layer):
    def __init__(self, **kwargs):
        # Remove 'trainable' and 'dtype' if present
        kwargs.pop('trainable', None)
        kwargs.pop('dtype', None)
        super().__init__(**kwargs)  # ← Now safe

        self.attn = layers.MultiHeadAttention(4, 128//4)
        self.norm1 = layers.LayerNormalization()
        self.ff1 = layers.Dense(128, activation='relu')
        self.ff2 = layers.Dense(128)
        self.norm2 = layers.LayerNormalization()

    def call(self, x, training=False):
        attn_out = self.attn(x, x)
        x = self.norm1(x + attn_out)
        ff_out = self.ff2(self.ff1(x))
        x = self.norm2(x + ff_out)
        return x

# === pos_emb ===
pos_emb = tf.Variable(tf.zeros([1, 1, 128]), trainable=True, name="pos_emb")

# === Custom Objects ===
CUSTOM_OBJECTS = {
    'TransformerBlock': TransformerBlock,
    'pos_emb': pos_emb
}

print("FIXED: TransformerBlock ready for loading!")
print("TransformerBlock FIXED — ready for loading!")

FIXED: TransformerBlock ready for loading!
TransformerBlock FIXED — ready for loading!


In [33]:
inputs = layers.Input((64, 64, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Reshape((1, 128))(x)

# Add pos_emb (external variable)
x = x + pos_emb

# Apply transformer
x = transformer_block(x)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(2, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_5 (Rescaling)         │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_5 (Reshape)             │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ add_8 (Add)                     │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ (None, 1, 128)         │        99,584 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_4      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 193,090 (754.26 KB)

 Trainable params: 193,090 (754.26 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
gen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

train_gen = gen.flow_from_directory(
    BINARY_DIR,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = gen.flow_from_directory(
    BINARY_DIR,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

print(f"Train: {train_gen.samples} | Val: {val_gen.samples}")

Found 5619 images belonging to 2 classes.
Found 1404 images belonging to 2 classes.
Train: 5619 | Val: 1404


In [35]:
print("Training...")
model.fit(train_gen, epochs=5, validation_data=val_gen, verbose=1)

acc = model.evaluate(val_gen, verbose=0)[1]
print(f"\nValidation Accuracy: {acc:.4f}")

Training...
Epoch 1/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.8416 - loss: 0.4019 - val_accuracy: 0.8882 - val_loss: 0.3194
Epoch 2/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - accuracy: 0.9345 - loss: 0.1912 - val_accuracy: 0.9074 - val_loss: 0.2371
Epoch 3/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.9413 - loss: 0.1644 - val_accuracy: 0.9081 - val_loss: 0.2297
Epoch 4/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.9511 - loss: 0.1427 - val_accuracy: 0.8476 - val_loss: 0.3783
Epoch 5/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.9564 - loss: 0.1226 - val_accuracy: 0.9010 - val_loss: 0.2313

Validation Accuracy: 0.9010


In [37]:
model.save("beta_model.keras")
print("Model saved as 'beta_model.keras'")


Model saved as 'beta_model.keras'
